# Setup Environment

## Setup Miniconda

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

## Check Install Directories

In [ ]:
!which conda # should return /usr/local/bin/conda

In [ ]:
!conda --version # should return 4.5.4

In [ ]:
!which python # still returns /usr/local/bin/python

In [ ]:
!python --version # now returns Python 3.6.5 :: Anaconda, Inc.

## Setup Environment using Miniconda

In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

In [ ]:
!conda --version # now returns 4.8.3

In [ ]:
!python --version # now returns Python 3.6.10 :: Anaconda, Inc.

In [ ]:
import sys
sys.path

In [ ]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

## Install Conda Packages

In [ ]:
!conda install --channel conda-forge featuretools --yes

In [ ]:
!conda install cudatoolkit=10.1 -y
!conda install pytorch -c pytorch -y
!conda install libsndfile=1.0.28 -c conda-forge -y
!conda install ipykernel -y

In [ ]:
!pip3 install absl-py librosa soundfile matplotlib scipy scikit-learn numba jiwer unidecode deepspeech==0.8.2

# Download and Check Dataset

## Download Dataset

In [ ]:
gdrive = True
if gdrive:
    # Download from Google Drive, much faster than from source
    # NOTE: Still getting this too work...
    !EEG_DATA_ID="1GT_hZTCx0ihdjcPvQFgYltmNZU5iIxvj" && \
    wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id=${EEG_DATA_ID}" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=${EEG_DATA_ID}" -O emg_data.tar.gz && rm -rf /tmp/cookies.txt
else:
    # Download from source, slower but should be 100% reliable
    !wget -O emg_data.tar.gz https://zenodo.org/record/4064409/files/emg_data.tar.gz?download=1

## Phoneme Dataset

In [ ]:
!wget -O text_alignments.tar.gz https://github.com/dgaddy/silent_speech_alignments/raw/main/text_alignments.tar.gz
!tar -xvf text_alignments.tar.gz

## Check EEG Dataset Size (Should be ~6.3GB)

In [ ]:
!du -sh emg_data

# Clone GDaddy/silent_speech Repository

In [ ]:
!git clone https://github.com/dgaddy/silent_speech/

# Closed Vocab Dataset
This dataset is split into two modes:
1. Voiced speech
2. Silent speech

In [ ]:
# Required
!pip install praat-textgrids

# Use this to change dir (If not already in silent_speech repo dir)
!pwd
switch = False
if switch:
    %cd silent_speech

## Analyse Entire Dataset Statistics

### Dataset Statistics
According to the [Digital Voicing of Silent Speech](https://arxiv.org/pdf/2010.02960.pdf)
paper, the closed vocabulary dataset should have the following chracteristics:
- Parallel silent / vocalised Speech (Es, Ev, Av)
  - 26 minutes silent / 30 minutes vocalised
  - Single session (data was only recorded once)
  - 500 utterances (500 different recordings)
  - Average of 4 words per utterance
  - 67 words in vocabulary (67 unique words were uttered in the entire dataset)
  
From the cells below, we can see the dataset matches the reported
statistics within the paper, aside from the vocabulary.

However, this is most likely due to how the vocabulary words
are calculated by the authors. Hopefully this is easy to resolve.

#### Extract Dataset Information

In [ ]:
#%cd silent_speech_analysis
!pwd

import os, json, glob

# Silent and voiced directories
emg_data_dir = "emg_data/"
silent_dir = os.path.join(os.getcwd(), emg_data_dir, "closed_vocab/silent/5-19_silent")
voiced_dir = os.path.join(os.getcwd(), emg_data_dir, "closed_vocab/voiced/5-19")

# Calc unique words within a data directory
def vocab_info(data_dir):
    path = os.path.join(data_dir)
    jsons = list(filter(lambda x: x.endswith(".json"), os.listdir(path)))
    infos = []
    for fname in jsons:
        cur_fname = os.path.join(data_dir, fname)
        with open(cur_fname) as f:
            info = json.loads(f.read())
            infos.append(info)
    return infos

# Get silent mode vocab
silent_info = vocab_info(silent_dir)
voiced_info = vocab_info(voiced_dir)

#### Calculate Vocabulary Size

In [ ]:
import re

reduce_year_vocab_size = False

#silent_vocab = set()
silent_vocab = list()
for info in silent_info:
    words = re.split(":| ", info["text"])
    for w in words:
        if len(w) == 4 and w.isnumeric() and reduce_year_vocab_size:
            w1 = w[0:2]
            w2 = w[2:4]
            silent_vocab.append(w1)
            silent_vocab.append(w2)
            #silent_vocab.add(w1)
            #silent_vocab.add(w2)
        else:
            # silent_vocab.add(w)
            silent_vocab.append(w)
silent_vocab.remove("")

voiced_vocab = set()
for info in voiced_info:
    words = re.split(":| ", info["text"])
    for w in words:
        if len(w) == 4 and w.isnumeric() and reduce_year_vocab_size:
            w1 = w[0:2]
            w2 = w[2:4]
            voiced_vocab.add(w1)
            voiced_vocab.add(w2)
        else:
            voiced_vocab.add(w)
voiced_vocab.remove("")

print('Silent Vocab Size, Voiced Vocab Size:', len(silent_vocab), len(voiced_vocab))

silent_vocab

I'm not sure how the authors calculated the 67 words in their vocab, as there are actually 113 unique words within the dataset. The way they calculate their vocabulary must be different to how I have. The method I have used is to sum 12 hour suffixes (AM/PM) with 1 digit number occurances, two digit number occurances, 7 days a week and 12 months a year.

The authors must be reducing the vocabulary down by grouping some of the numbers which are uttered, further investigation here is required.

#### Utterance Calculations

In [ ]:
raw_silent_utters = [info["text"] for info in silent_info]
raw_silent_utters = list(filter(lambda x: x != "", raw_silent_utters))

raw_voiced_utters = [info["text"] for info in voiced_info]
raw_voiced_utters = list(filter(lambda x: x != "", raw_voiced_utters))

silent_utterances = set(raw_silent_utters)
voiced_utterances = set(raw_voiced_utters)

silent_utterance_indices = set([info["sentence_index"] for info in silent_info])
voiced_utterance_indices = set([info["sentence_index"] for info in voiced_info])

def get_utter_word_count(utterance):
    words = re.split(":| ", utterance)
    cnt = 0
    for w in words:
        if len(w) == 4 and w.isnumeric():
            cnt += 2
        else:
            cnt += 1
    return cnt

avg_silent_utter_words = [get_utter_word_count(u) for u in raw_silent_utters]
print('Silent utter avg:', set(avg_silent_utter_words), avg_silent_utter_words, sum(avg_silent_utter_words), len(avg_silent_utter_words), sum(avg_silent_utter_words) / len(avg_silent_utter_words))
avg_silent_utter_words = sum(avg_silent_utter_words) / len(avg_silent_utter_words)

avg_voiced_utter_words = [get_utter_word_count(u) for u in raw_voiced_utters]
print('Voiced utter avg:', set(avg_voiced_utter_words), avg_voiced_utter_words, sum(avg_voiced_utter_words), len(avg_voiced_utter_words), sum(avg_voiced_utter_words) / len(avg_voiced_utter_words))
avg_voiced_utter_words = sum(avg_voiced_utter_words) / len(avg_voiced_utter_words)

print('Silent, Voiced utterance counts:', len(silent_utterances), len(voiced_utterances))
print('Index counts (silent, voiced):', len(silent_utterance_indices), len(voiced_utterance_indices))
print('Raw (silent, voiced) utterance counts:', len(raw_silent_utters), len(raw_voiced_utters))
print('Avg words/utterance (silent, voiced) counts:', avg_silent_utter_words, avg_voiced_utter_words)

The sentence index (in other words, the indices into the utterances) matches what is reported exactly.
The [Digital Voicing of Silent Speech](https://arxiv.org/pdf/2010.02960.pdf) paper reports
500 utterances, and their are 500 sentences (i.e. utterances) plus an additional recording which
contains no speech.
Presumably this is an initial recording to ensure the recording environment (OpenBCI
EEG recording hardware and software) were working correctly.

#### Audio Calculations

In [163]:
import os

"""
silent_dir
voiced_dir
"""

import librosa
silent_flacs = list(filter(lambda x: x.endswith("clean.flac"), os.listdir(silent_dir)))
voiced_flacs = list(filter(lambda x: x.endswith("clean.flac"), os.listdir(voiced_dir)))

silent_audio_duration = 0 # Seconds
for fname in silent_flacs:
    silent_audio_duration += librosa.get_duration(filename=os.path.join(silent_dir, fname))
    
voiced_audio_duration = 0 # Seconds
for fname in voiced_flacs:
    voiced_audio_duration += librosa.get_duration(filename=os.path.join(voiced_dir, fname))
    
silent_mins = silent_audio_duration / 60.0
audio_mins = voiced_audio_duration / 60.0

print(f'Audio (Silent, Voiced) duration (mins): {silent_mins:.2f}, {audio_mins:.2f}')

Audio (Silent, Voiced) duration (mins): 26.46, 29.53
